In [11]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > .5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    if len(faces) > 0:
        preds = maskNet.predict(faces)
    return (len(faces),locs, preds)



prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
weightsPath = os.path.sep.join(["face_detector","res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model("mask_type.h5")

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box


        lst = ["Hanker_mask","N-95_mask","No_mask","Three_layer_mask"]
        label = lst[np.argmax(pred)]
        color = (0, 255, 0)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label,max(pred))

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading face mask detector model...
[INFO] starting video stream...
200
0 [164.05611038 138.26344013 232.61108398 232.93790817]
1
200
0 [161.47601604 139.21827078 232.44111538 233.86362791]
1
200
0 [160.7411623  138.49387765 232.27965832 233.30810666]
1
200
0 [161.73288822 136.36413217 232.25877285 232.18351007]
1
200
0 [162.73676157 139.17971849 231.98828697 232.80755281]
1
200
0 [161.0450983  138.62184584 232.68690109 232.49613047]
1
200
0 [161.70595884 139.63472843 232.29675293 231.73831701]
1
200
0 [163.00926208 139.71873522 232.34038353 231.46494627]
1
200
0 [161.22024059 138.83972168 232.48710632 233.23777914]
1
200
0 [161.30948067 139.60757554 232.22682476 229.48544025]
1
200
0 [161.24032736 138.94239664 231.93354607 231.25280142]
1
200
0 [160.04232168 138.15860152 232.74893761 231.88617826]
1
200
0 [159.86530781 138.52419555 232.48491287 230.33527136]
1
200
0 [161.63730621 137.70472705 233.11009407 232.83709288]
1
200
0 [162.52288818 138.56193423 232.70218372 231.7325055